# Multiple Custom Grouping Aggregations

This challenge is going to be fairly difficult, but should answer a question that many pandas users face - What is the best way to do a grouping operation that does many custom aggregations? In this context, a 'custom aggregation' is defined as one that is not directly available to use from pandas and one that you must write a custom function for. 

In Pandas Challenge 1, a single aggregation, which required a custom grouping function, was the desired result. In this challenge, you'll need to make several aggregations when grouping. There are a few different solutions to this problem, but depending on how you arrive at your solution, there could arise enormous performance differences. I am looking for a compact, readable solution with very good performance.

### Sales Data

In this challenge, you will be working with some mock sales data found in the sales.csv file. It contains 200,000 rows and 9 columns.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/sales.csv', parse_dates=['date'])
df.head()

,customer_id,date,country,region,delivery_type,cost_type,duration,revenue,cost
0,13763,2019-03-25,Portugal,F,slow,expert,60,553,295
1,13673,2019-12-06,Singapore,I,slow,experienced,60,895,262
2,10287,2018-09-04,India,I,slow,novice,60,857,260
3,14298,2018-06-21,Morocco,F,fastest,expert,120,741,238
4,11523,2019-01-05,Luxembourg,A,fast,expert,120,942,263


In [3]:
df.shape

(200000, 9)

### Challenge

There are many aggregations that you will need to return and it will take some time to understand what they are and how to return them. The following definitions for two time periods will be used throughout the aggregations.

Period **2019H1** is defined as the time period beginning January 1, 2019 and ending June 30, 2019.
Period **2018H1** is defined as the time period beginning January 1, 2018 and ending June 30, 2018.

### Aggregations
Now, I will list all the aggregations that are expected to be returned. Each bullet point represents a single column. Use the first word after the bullet point as the new column name.

For every country and region, return the following:
* recency: Number of days between today's date (9/9/2019) and the maximum value of the 'date' column 
* fast_and_fastest: Number of unique customer_id in period 2019H1 with delivery_type either 'fast' or 'fastest'
* rev_2019: Total revenue for the period 2019H1
* rev_2018: Total revenue for the period 2018H1
* cost_2019: Total cost for period 2019H1
* cost_2019_exp: Total cost for period 2019H1 with cost_type 'expert'
* other_cost: Difference between cost_2019 and cost_2019_exp
* rev_per_60: Total of revenue when duration equals 60 in period 2019H1 divided by number of unique customer_id when duration equals 60 in period 2019H1 
* profit_margin: Take the difference of rev_2019 and cost_2019_exp then divide by rev_2019. Return as percentage
* cost_exp_per_60: Total of cost when duration is 60 and cost_type is 'expert' in period 2019H1 divided by the number of unique customer_id when duration equals 60 and cost_type is 'expert' in period 2019H1 
* growth: Find the percentage growth from revenue in period 2019H1 compared to the revenue in period 2018H1

In [4]:
H1_2019_MIN = pd.to_datetime('01-01-2019')
H1_2019_MAX = pd.to_datetime('30-06-2019')
H1_2018_MIN = pd.to_datetime('01-01-2018')
H1_2018_MAX = pd.to_datetime('30-06-2018')
TODAY = pd.to_datetime('09-09-2019')

GROUPBY_COLS = ['country', 'region']

df['date'] = pd.to_datetime(df['date'])

PERIOD_2019 = (df['date'] >= H1_2019_MIN) & (df['date'] <= H1_2019_MAX)
PERIOD_2018 = (df['date'] >= H1_2018_MIN) & (df['date'] <= H1_2018_MAX)

In [5]:
# recency: Number of days between today's date (9/9/2019) and the maximum value of the 'date' column
(df
 .groupby(GROUPBY_COLS)
 .agg({'date': 'max'})
 .sub(TODAY)
 .head())

date
country   region        
Argentina A      86 days
          B      88 days
          C      88 days
          D      88 days
          E      88 days

In [6]:
# fast_and_fastest: Number of unique customer_id in period 2019H1 with delivery_type either 'fast' or 'fastest'
condition = PERIOD_2019 & (df['delivery_type'].isin(['fast', 'fastest']))
(df.loc[condition]
 .groupby(GROUPBY_COLS)
 .agg({'customer_id': 'nunique'})
 .head())

customer_id
country   region             
Argentina A                74
          B                61
          C                56
          D                66
          E                92

In [7]:
# rev_2019: Total revenue for the period 2019H1
condition = PERIOD_2019
(df.loc[condition]
 .groupby(GROUPBY_COLS)
 .agg({'revenue': 'sum'})
 .head())

revenue
country   region         
Argentina A        150508
          B        139048
          C        118035
          D        131728
          E        146201

In [8]:
# rev_2018: Total revenue for the period 2018H1
condition = PERIOD_2018
(df.loc[condition]
 .groupby(GROUPBY_COLS)
 .agg({'revenue': 'sum'})
 .head())

revenue
country   region         
Argentina A         82912
          B         92112
          C         98472
          D         79600
          E         93119

In [9]:
# cost_2019: Total cost for period 2019H1
condition = PERIOD_2019
(df.loc[condition]
 .groupby(GROUPBY_COLS)
 .agg({'cost': 'sum'})
 .head())

cost
country   region       
Argentina A       49577
          B       46153
          C       38786
          D       44190
          E       49600

In [10]:
# cost_2019_exp: Total cost for period 2019H1 with cost_type 'expert'
condition = PERIOD_2019 & (df['cost_type'] == 'expert')
(df.loc[condition]
 .groupby(GROUPBY_COLS)
 .agg({'cost': 'sum'})
 .head())

cost
country   region       
Argentina A       18553
          B       15732
          C       12661
          D       17217
          E       18372

In [11]:
# other_cost: Difference between cost_2019 and cost_2019_exp
# cost_2019.sub(cost_2019_exp)

In [12]:
# rev_per_60: Total of revenue when duration equals 60 in period 2019H1 
# divided by number of unique customer_id when duration equals 60 in period 2019H1

condition = PERIOD_2019 & (df['duration'] == 60)

result = df.loc[condition].groupby(GROUPBY_COLS).agg(dict(revenue='sum',
                                                          customer_id='nunique'))
(result['revenue'] / result['customer_id']).to_frame('rev_per_60').head()

rev_per_60
country   region            
Argentina A       772.891304
          B       749.677778
          C       780.013514
          D       730.469880
          E       747.173469

In [13]:
# profit_margin: Take the difference of rev_2019 and cost_2019_exp then divide by rev_2019. Return as percentage

# rev_2019.sub(cost_2019_exp).div(rev_2019).multiply(100).round(2).astype('str') + '%'

In [14]:
# cost_exp_per_60: Total of cost when duration is 60 and cost_type is 'expert' in period 2019H1 
# divided by the number of unique customer_id when duration equals 60 and cost_type is 'expert' in period 2019H1

condition = PERIOD_2019 & (df['cost_type'] == 'expert')
result = (df.loc[condition]
          .groupby(GROUPBY_COLS)
          .agg(dict(cost='sum',
                    customer_id='nunique')))
(result['cost']
 .div(result['customer_id'])
 .to_frame('cost_exp_per_60')
 .head())

cost_exp_per_60
country   region                 
Argentina A            250.716216
          B            249.714286
          C            243.480769
          D            249.521739
          E            255.166667

In [15]:
# growth: Find the percentage growth from revenue in period 2019H1 compared to the revenue in period 2018H1
revenue_2019 = df.loc[PERIOD_2019].groupby(GROUPBY_COLS).agg(dict(revenue='sum'))
revenue_2018 = df.loc[PERIOD_2018].groupby(GROUPBY_COLS).agg(dict(revenue='sum'))

revenue_2019.div(revenue_2018).head()

revenue
country   region          
Argentina A       1.815274
          B       1.509554
          C       1.198666
          D       1.654874
          E       1.570045

# Become a pandas expert

If you are looking to completely master the pandas library and become a trusted expert for doing data science work, check out my book [Master Data Analysis with Python][1]. It comes with over 300 exercises with detailed solutions covering the pandas library in-depth.

[1]: https://www.dunderdata.com/master-data-analysis-with-python